In [17]:
import csv
import numpy as np

wine_path = "./winequality-white.csv" ## 数据集的路径

## delimiter 表示数据之间的分隔符
## 在这里指定二维数组的类型 和用于分割每一行各个值的分隔符  并且skiprows表示不读取第一行（列名）
wineq_numpy = np.loadtxt(wine_path,dtype = np.float32,delimiter=";",skiprows=1)  ## 加载数据 转换为numpy张量

wineq_numpy

array([[ 7.  ,  0.27,  0.36, ...,  0.45,  8.8 ,  6.  ],
       [ 6.3 ,  0.3 ,  0.34, ...,  0.49,  9.5 ,  6.  ],
       [ 8.1 ,  0.28,  0.4 , ...,  0.44, 10.1 ,  6.  ],
       ...,
       [ 6.5 ,  0.24,  0.19, ...,  0.46,  9.4 ,  6.  ],
       [ 5.5 ,  0.29,  0.3 , ...,  0.38, 12.8 ,  7.  ],
       [ 6.  ,  0.21,  0.38, ...,  0.32, 11.8 ,  6.  ]], dtype=float32)

In [18]:
## csv.reader返回一个reader对象，使用该对象遍历csv文件中的行  next返回迭代器中的下一个项目
col_list = next(csv.reader(open(wine_path),delimiter=';'))  ## 这里读取的是第一行
print(wineq_numpy.shape)  ## 打印形状 二维 
col_list  ## 输出第一行 所有列名

(4898, 12)


['fixed acidity',
 'volatile acidity',
 'citric acid',
 'residual sugar',
 'chlorides',
 'free sulfur dioxide',
 'total sulfur dioxide',
 'density',
 'pH',
 'sulphates',
 'alcohol',
 'quality']

In [19]:
## 将numpy数组转换成张量
import torch
wineq = torch.from_numpy(wineq_numpy)  
wineq.shape,wineq.type()  ## 这里的shape 也是一个张量形式

(torch.Size([4898, 12]), 'torch.FloatTensor')

In [20]:
data = wineq[:,:-1]  ## 除去最后一列之外的所有列  所有行
print(data)
print(data.shape)

tensor([[ 7.0000,  0.2700,  0.3600,  ...,  3.0000,  0.4500,  8.8000],
        [ 6.3000,  0.3000,  0.3400,  ...,  3.3000,  0.4900,  9.5000],
        [ 8.1000,  0.2800,  0.4000,  ...,  3.2600,  0.4400, 10.1000],
        ...,
        [ 6.5000,  0.2400,  0.1900,  ...,  2.9900,  0.4600,  9.4000],
        [ 5.5000,  0.2900,  0.3000,  ...,  3.3400,  0.3800, 12.8000],
        [ 6.0000,  0.2100,  0.3800,  ...,  3.2600,  0.3200, 11.8000]])
torch.Size([4898, 11])


In [21]:
target = wineq[:,-1]  ## 保存分数一列
print(target)   ## 一维张量
print(target.shape)

tensor([6., 6., 6.,  ..., 6., 7., 6.])
torch.Size([4898])


In [22]:
## 将target 转换成标签张量  

## 第一种选择将标签看成整数向量

target = wineq[:,-1].long()
print(target)

tensor([6, 6, 6,  ..., 6, 7, 6])


In [23]:
## 独热编码
target_onehot = torch.zeros(target.shape[0],10)  ## 创建新的张量 全为0 一共有十列

## pytorch中 带有下划线结尾的方法 不会返回新的张量

## scatter_参数
## 指定后面两个参数要处理的维度   每一行
## 列张量 指示要填充的索引
## 包含要填充元素的张量或者单个标量（1.0）

## 索引张量  必须要有和待填充张量相同的维数  unsqueeze(1) 插入新的一维
target_onehot.scatter_(1,target.unsqueeze(1),1.0)
print(target_onehot)

## 对于每一行 获取目标标签的索引 并将其用作列索引以设置值1.0

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 1., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])


In [24]:
## 计算样本的均值  计算每一列的均值和标准差：
data_mean = torch.mean(data,dim = 0)
print(data_mean)## 十一列  张量


tensor([6.8548e+00, 2.7824e-01, 3.3419e-01, 6.3914e+00, 4.5772e-02, 3.5308e+01,
        1.3836e+02, 9.9403e-01, 3.1883e+00, 4.8985e-01, 1.0514e+01])


In [25]:
data_var = torch.var(data,dim = 0) ## 计算每一列的方差
print(data_var)

tensor([7.1211e-01, 1.0160e-02, 1.4646e-02, 2.5726e+01, 4.7733e-04, 2.8924e+02,
        1.8061e+03, 8.9455e-06, 2.2801e-02, 1.3025e-02, 1.5144e+00])


In [27]:
## dim = 0 表示沿着维数0进行计算  通过减去平均值并且除以标准差来对数据进行归一化  归一化有助于神经网络的学习过程
data_normalized = (data - data_mean) / torch.sqrt(data_var)
data_normalized

tensor([[ 1.7208e-01, -8.1761e-02,  2.1326e-01,  ..., -1.2468e+00,
         -3.4915e-01, -1.3930e+00],
        [-6.5743e-01,  2.1587e-01,  4.7996e-02,  ...,  7.3995e-01,
          1.3422e-03, -8.2419e-01],
        [ 1.4756e+00,  1.7450e-02,  5.4378e-01,  ...,  4.7505e-01,
         -4.3677e-01, -3.3663e-01],
        ...,
        [-4.2043e-01, -3.7940e-01, -1.1915e+00,  ..., -1.3130e+00,
         -2.6153e-01, -9.0545e-01],
        [-1.6054e+00,  1.1666e-01, -2.8253e-01,  ...,  1.0049e+00,
         -9.6251e-01,  1.8574e+00],
        [-1.0129e+00, -6.7703e-01,  3.7852e-01,  ...,  4.7505e-01,
         -1.4882e+00,  1.0448e+00]])

In [28]:
## torch.le 函数确定target中哪些行对应的分数小于或者等于3
bad_indexes = torch.le(target,3)  ## 小于或者等于3 的被标记成1 其余的是0
print(bad_indexes.shape)
print(bad_indexes.dtype)
print(bad_indexes.sum())

torch.Size([4898])
torch.bool
tensor(20)


In [30]:
## 通过使用pytorch中的高级索引功能，可以使用0/1张量来索引数据张量 
## 此张量本质上将数据筛选为仅为索引张量中1对应的元素或者行
bad_data = data[bad_indexes]
print(bad_data.shape)  ## 将bad_indexes作为索引张量 筛选出对应的行 列数保持不变


torch.Size([20, 11])


In [32]:
## 将data分成 好 中 坏三类葡萄酒信息
## 对每一列取均值
bad_data = data[torch.le(target,3)]  ## 先用le 筛选出符合条件的索引张量 然后取出对应的行
mid_data = data[torch.gt(target,3) & torch.lt(target,7)] 
good_data = data[torch.ge(target,7)] ## 筛选出好的数据

## 计算好 中 坏 每一列的均值
bad_mean = torch.mean(bad_data,dim=0)
mid_mean = torch.mean(mid_data,dim=0)
good_mean = torch.mean(good_data,dim = 0)

## zip将这些打印的参数全部封装到args中 然后使用*args进行取出
for i,args in enumerate(zip(col_list,bad_mean,mid_mean,good_mean)):
    print('{:2} {:20} {:6.2f} {:6.2f} {:6.2f}'.format(i,*args))  ## 打印

 0 fixed acidity          7.60   6.89   6.73
 1 volatile acidity       0.33   0.28   0.27
 2 citric acid            0.34   0.34   0.33
 3 residual sugar         6.39   6.71   5.26
 4 chlorides              0.05   0.05   0.04
 5 free sulfur dioxide   53.33  35.42  34.55
 6 total sulfur dioxide 170.60 141.83 125.25
 7 density                0.99   0.99   0.99
 8 pH                     3.19   3.18   3.22
 9 sulphates              0.47   0.49   0.50
10 alcohol               10.34  10.26  11.42


In [36]:
## 将二氧化硫的含量作为区分好酒还是坏酒的标准  取出低于标准值的全部索引
total_sulfur_threshold = 141.83
total_sulfur_data = data[:,6] ## 取出所有数据的所有行 第7列
print(total_sulfur_data)
predicted_indexes = torch.lt(total_sulfur_data,total_sulfur_threshold)
print(predicted_indexes.shape)
print(predicted_indexes.dtype)
print(predicted_indexes.sum())

tensor([170., 132.,  97.,  ..., 111., 110.,  98.])
torch.Size([4898])
torch.bool
tensor(2727)


In [37]:
## 获取实际优质葡萄酒的索引
actual_indexes = torch.gt(target,5)
print(actual_indexes.shape)
print(actual_indexes.dtype)
print(actual_indexes.sum())

torch.Size([4898])
torch.bool
tensor(3258)


In [38]:
n_matches = torch.sum(actual_indexes & predicted_indexes).item()
n_predicted = torch.sum(predicted_indexes).item()
n_actual = torch.sum(actual_indexes).item()

print(n_matches)
print(n_matches / n_predicted)
print(n_matches / n_actual)

2018
0.74000733406674
0.6193984039287906
